Chapter 7 Quantum Cryptography

In [ ]:
#Caesar's cipher 
message = "YES"

shift = 3 

encryption = ""

for c in message:

    # check if character is an uppercase letter
    if c.isupper():

        # find the position in 0-25
        c_unicode = ord(c)

        c_index = ord(c) - ord("A")

        # perform the shift
        new_index = (c_index + shift) % 26

        # convert to new character
        new_unicode = new_index + ord("A")

        new_character = chr(new_unicode)

        # append to encrypted string
        encryption = encryption + new_character

    else:

        # since character is not uppercase, leave it as it is
        encryption += c
        
print("Plain text:", message)

print("Encrypted text:", encryption)

In [ ]:
# One Time Pad (Vernmam Cipher)

key = 5
message = "HELLO"
encrypt = ""

for i in range(len(message)):
        letter = ord(message[i])-65      # Letters now range 0-25
        letter = (letter + key)%25 # Alphanumeric + key mod 25 = 0-25
        letter +=65
        

        encrypt = encrypt + chr(letter)

print("Original message is:", message)        
print("Encrypted message is:", encrypt)

In [ ]:
# A Diffie-Hellman Key Exchange Scheme

import math 

p = input("Enter the shared prime number: \n")
p = int(p)
g = input("Enter the shared base: \n")
g = int(g)


a = input("Enter Alice's secret key: \n")
a = int(a)
b = input("Enter Bob's secret key: \n")
b = int(b)

AlicePublicKey = (g ** a ) % p
BobPublicKey = (g ** b) % p

AliceSecret = (BobPublicKey ** a) % p
BobSecret = (AlicePublicKey ** b) % p

print(AliceSecret)
print(BobSecret)

In [ ]:
#A simple pseudorandom number generator.

import random

random.seed(42) 

x = []
y = [] 

for i in range(10):
  a = random.random()
  x.append(a)

print(x)

for i in range(5):
  b = random.randint(0,4)
  y.append(b)

print(y)

In [ ]:
# MD5 hashing

import hashlib

AliceMessage = hashlib.md5()
Alice = "This is Alice"
Alice = Alice.encode(encoding='utf-8')
AliceMessage.update(Alice)
print("Alice's MD5 digest is: \n", AliceMessage.hexdigest())

print("Alice's digest size is: \n", AliceMessage.digest_size)
print("Alice's block size is: \n", AliceMessage.block_size)

BobMessage = hashlib.md5()
Bob = "This is Bob"
Bob = Bob.encode(encoding='utf-8')
BobMessage.update(Bob)
print("Bob's MD5 digest is: \n", BobMessage.hexdigest())

print("Bob's digest size is: \n", BobMessage.digest_size)
print("Bob's block size is: \n", BobMessage.block_size)

In [ ]:
# SHA-512 hashing

import hashlib

AliceMessage = hashlib.sha3_512()
Alice = "This is Alice"
Alice = Alice.encode(encoding='utf-8')
AliceMessage.update(Alice)
print("Alice's SHA digest is: \n", AliceMessage.hexdigest())

print("Alice's digest size is: \n", AliceMessage.digest_size)
print("Alice's block size is: \n", AliceMessage.block_size)

BobMessage = hashlib.sha3_512()
Bob = "This is Bob"
Bob = Bob.encode(encoding='utf-8')
BobMessage.update(Bob)
print("Bob's SHA digest is: \n", BobMessage.hexdigest())

print("Bob's digest size is: \n", BobMessage.digest_size)
print("Bob's block size is: \n", BobMessage.block_size)

In [ ]:
# 5-qubit QRNG 

#!pip install qiskit 

import random
from qiskit import *
from qiskit.visualization import plot_histogram

random.seed(42)

circuit = QuantumCircuit(5,5)

circuit.h(0)

circuit.x(1)
circuit.h(1)

circuit.h(2)

circuit.x(3)
circuit.h(3)

circuit.h(4)

circuit.measure([0,1,2,3,4], [0,1,2,3,4])
circuit.draw(output='text')

simulator = Aer.get_backend('qasm_simulator')
result = execute(circuit, backend=simulator, shots=1024).result()
plot_histogram(result.get_counts(circuit))

In [ ]:
#Implementation of the BB84 Protocol

from qiskit import QuantumCircuit, execute, Aer 
from qiskit.visualization import *
#from qiskit.tools.monitor import *
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

circ = QuantumCircuit(1,1)
circ.x(0)
circ.barrier()
circ.h(0) 
circ.barrier()
circ.measure(0,0) 
circ.barrier()
circ.draw(output='mpl')



# Alice generates n random bits (some of these bits will form the key)
backend = Aer.get_backend('qasm_simulator')
result = execute(circ, backend, shots=128, memory = True).result()
bits_alice = [int(q) for q in result.get_memory()] 
print(bits_alice)

# Alice randomly chooses the bases in which she is going to measure
#backend = Aer.get_backend('qasm_simulator')
result = execute(circ, backend, shots=128, memory = True).result()
basis_alice = [int(q) for q in result.get_memory()] 
print(basis_alice)

# Bob also chooses at random the bases in which he will measure
result = execute(circ, backend, shots=128, memory = True).result()
basis_bob = [int(q) for q in result.get_memory()] 
print(basis_bob)

# Now, Alice codes each bit of her initial string as a qubit and sends it to Bob, who measures in his basis

bits_bob = []

for i in range(128):
    circ_send = QuantumCircuit(1,1)
    if bits_alice[i]:  
        circ_send.x(0)
    if basis_alice[i]: 
        circ_send.h(0)
        
    # Alice sends the qubit to Bob and he measures
    
    if basis_bob[i]: 
        circ_send.h(0) 
    
    circ_send.measure(0,0)
    
    result = execute(circ_send, backend, shots = 1, memory = True).result()
    bits_bob.append(int(result.get_memory()[0]))
    
print(bits_bob)

# Bob tells Alice the basis he used for his measurements 
# Alice confirms which of the basis are correct

key = []

for i in range(128):
    if basis_alice[i] == basis_bob[i]:
        key.append(bits_bob[i])
        
print("Key length", len(key))
print(key)

In [ ]:
#The implementation of the B92 protocol

from qiskit import QuantumCircuit, execute, Aer 
from qiskit.visualization import *
#from qiskit.tools.monitor import *
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

circ = QuantumCircuit(1,1)
circ.x(0)
circ.barrier()
circ.h(0) 
circ.barrier()
circ.measure(0,0) 
circ.barrier()
circ.draw(output='text')

n = 128

# Alice generates n random bits (some of these bits will form the key)
backend = Aer.get_backend('qasm_simulator')
result = execute(circ, backend, shots=n, memory = True).result()
bits_alice = [int(q) for q in result.get_memory()] 
print(bits_alice)

# Bob also chooses at random the bases in which he will measure
result = execute(circ, backend, shots=n, memory = True).result()
basis_bob = [int(q) for q in result.get_memory()] 
print(basis_bob)

bits_bob = []
for i in range(n):
    circ_send = QuantumCircuit(1,1)
    if bits_alice[i] == 0:  #Don't change the basis
        circ_send.id(0)
    if bits_alice[i] == 1: #Change the basis
        circ_send.h(0)
    else:
      circ_send.id(0)
    circ_send.measure(0,0)
    
    result = execute(circ_send, backend, shots = 1, memory = True).result()
    bits_bob.append(int(result.get_memory()[0]))

print(bits_bob)

key = []

for i in range(n):
    if bits_alice[i] == bits_bob[i]:
        key.append(bits_bob[i])
        
print("Key length is:", len(key))
print("The secret Key is:", key)

In [ ]:
#Implementation of the E91 protocol
from qiskit import *
#from qiskit.visualization import *

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

A = [0, np.pi/8, np.pi/4] #choice of bases for Alice
B = [0, np.pi/8, -1*np.pi/8] #choice of bases for Bob
basesA = []
basesB = []
output = []

for i in range(100):
  
  circ = QuantumCircuit(2, 2)
  circ.h(0)
  circ.cx(0,1)  
  Ta = np.random.choice(A)
  Tb = np.random.choice(B)
  circ.rz(Ta, 0)
  circ.rz(Tb, 1)
  circ.measure([0, 1], [0, 1])
  #circ.draw(output = 'text')
  
  backend = Aer.get_backend('qasm_simulator')
  result = execute(circ, backend, shots=1, memory=True).result()
  value = result.get_memory()
  output.append(value)

print("The output is:", output)

In [ ]:
#postquantum implementation - NewHope

#!pip install pynewhope 

'''
The code is adopted from: 
https://pypi.org/project/PyNewHope/
'''
from pynewhope import newhope
import numpy as np

np.random.seed(42)

# Step 1: Alice generates random keys and her public msg to Bob
ka, ma = newhope.keygen()

# Step 2: Bob receives the msg from Alice and responds to Alice with a msg
skb, mb = newhope.sharedB(ma)

# Step 3: Alice receives the msg from Bob and generates her shared secret
ska = newhope.sharedA(mb, ka)

if ska == skb:
    print("\nSuccessful key exchange! Keys match.")
else:
    print("\nError! Keys do not match.")

print("The shared key is:", ska)

In [ ]:
#SPHINCS implementation - postquantum technique
#!pip install pyspx

'''
This code is adopted from: 
https://cryptobook.nakov.com/quantum-safe-cryptography/quantum-safe-signatures-example
'''

import pyspx.shake256_128s as sphincs
import os, binascii
import numpy as np

np.random.seed(42)

# Key generation: private + public key
seed = os.urandom(sphincs.crypto_sign_SEEDBYTES)
public_key, secret_key = sphincs.generate_keypair(seed)

# Sign message and verify signature
message = b'Hello World'
signature = sphincs.sign(message, secret_key)
valid = sphincs.verify(message, signature, public_key)

# Verify tampered message + signature
message = b'Hello World'
valid = sphincs.verify(message, signature, public_key)
print("Tampered message:", message)
print("Tampered signature valid?", valid)

message = b'Bye World'
valid = sphincs.verify(message, signature, public_key)
print("Tampered message:", message)
print("Tampered signature valid?", valid)